# **Clasterización**

En este archivo realizaremos una clasterización que verifica los siguientes puntos:
- Aprendizaje No-Supervisado en Python.
- Ingeniería de variables (Feature engineering).
- Centroid-based Clustering (K-Means , Mean-Shift & Mini-Batch K-Means).
- Density-based clustering (DBSCAN, OPTICS).
- Distribution-based clustering (GMM).
- Hierarchical clustering (Agglomerative Clustering).

Primero importamos todas las librerías que usaremos y las instalamos en caso de ser necesario.

**¿Cómo hacemos que el código sea de aprendizaje no supervisado?**

El aprendizaje no supervisado se aplica en este código a través de los algoritmos de clustering que se utilizan para agrupar los datos en grupos o clusters sin la necesidad de tener etiquetas de clase predefinidas. 

Estos algoritmos son ejemplos de técnicas de aprendizaje no supervisado que se utilizan para encontrar patrones y estructuras subyacentes en los datos sin necesidad de información previa sobre las clases a las que pertenecen los datos. El objetivo es agrupar los datos en grupos similares o "clústeres" basándose únicamente en la similitud entre las muestras. Cada algoritmo tiene su propio enfoque para realizar esta tarea, como la minimización de la distancia intra-cluster en el caso de K-Means o la detección de densidades locales en el caso de DBSCAN.

In [3]:
# Instalamos las librerías necesarias
%pip install pandas
%pip install numpy
%pip install sklearn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


Defaulting to user installation because normal site-packages is not writeable
  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
      
      More information is available at
      https://github.com/scikit-learn/sklearn-pypi-packag

In [4]:
# Importamos las librerías necesarias
import pandas as pd
from sklearn.cluster import KMeans, MeanShift, DBSCAN, OPTICS, AgglomerativeClustering
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import LabelEncoder, StandardScaler

El próximo paso es cargar los datos limpios.

In [5]:
datos = pd.read_csv('../data/equipos_limpio.csv')
datos.head()

,Season,Squad,Country,# Pl,Age,MP,Starts,Gls,Ast,G+A,G-PK,PK,PKatt,CrdY,CrdR
0,2022-2023,Ajax,Países bajos,19,26.4,6,66,11,9,20,10,1,1,15.0,1.0
1,2022-2023,Atlético Madrid,España,22,28.6,6,66,4,3,7,4,0,2,11.0,0.0
2,2022-2023,Barcelona,España,26,26.4,6,66,12,10,22,12,0,0,9.0,0.0
3,2022-2023,Bayern Munich,Alemania,24,26.6,10,110,21,19,40,20,1,1,20.0,1.0
4,2022-2023,Benfica,Portugal,24,26.0,10,110,25,16,41,20,5,5,19.0,0.0


Ahora comenzamos con un preprocesamiento de los datos que puede considerarse como una forma básica de ingeniería de variables. En nuestro caso es el manejo de valores faltantes, eliminación de columnas irrelevantes y el escalado de características.

In [6]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 640 entries, 0 to 639
Data columns (total 15 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Season   640 non-null    object 
 1   Squad    640 non-null    object 
 2   Country  640 non-null    object 
 3   # Pl     640 non-null    int64  
 4   Age      640 non-null    float64
 5   MP       640 non-null    int64  
 6   Starts   640 non-null    int64  
 7   Gls      640 non-null    int64  
 8   Ast      640 non-null    int64  
 9   G+A      640 non-null    int64  
 10  G-PK     640 non-null    int64  
 11  PK       640 non-null    int64  
 12  PKatt    640 non-null    int64  
 13  CrdY     576 non-null    float64
 14  CrdR     576 non-null    float64
dtypes: float64(3), int64(9), object(3)
memory usage: 75.1+ KB


Vemos que todavía tenemos algunas filas nulas. Al limpiar los datos no nos importaba tener algunas filas nulas, pero para hacer la clasterización es muy importante no contar con ningún dato de este tipo.

In [7]:
# Eliminamos las filas que contienen valores nulos
datos = datos.dropna()

# Vemos que se ha hecho el cambio correctamente
datos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 576 entries, 0 to 639
Data columns (total 15 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Season   576 non-null    object 
 1   Squad    576 non-null    object 
 2   Country  576 non-null    object 
 3   # Pl     576 non-null    int64  
 4   Age      576 non-null    float64
 5   MP       576 non-null    int64  
 6   Starts   576 non-null    int64  
 7   Gls      576 non-null    int64  
 8   Ast      576 non-null    int64  
 9   G+A      576 non-null    int64  
 10  G-PK     576 non-null    int64  
 11  PK       576 non-null    int64  
 12  PKatt    576 non-null    int64  
 13  CrdY     576 non-null    float64
 14  CrdR     576 non-null    float64
dtypes: float64(3), int64(9), object(3)
memory usage: 72.0+ KB


Observamos que hay algunas variables categóricas que podríamos pasar a numéricas, pues nos podrían ayudar en nuestra predicción posteriormente.

In [8]:
print(datos['Season'].unique())
print(datos['Squad'].unique())
print(datos['Country'].unique())

['2022-2023' '2021-2022' '2020-2021' '2019-2020' '2018-2019' '2016-2017'
 '2014-2015' '2013-2014' '2012-2013' '2011-2012' '2010-2011' '2009-2010'
 '2008-2009' '2007-2008' '2006-2007' '2005-2006' '2004-2005' '2003-2004']
['Ajax' 'Atlético Madrid' 'Barcelona' 'Bayern Munich' 'Benfica' 'Celtic'
 'Chelsea' 'Club Brugge' 'Dinamo Zagreb' 'Dortmund' 'Eint Frankfurt'
 'FC Copenhagen' 'Inter' 'Juventus' 'Leverkusen' 'Liverpool'
 'Maccabi Haifa' 'Manchester City' 'Marseille' 'Milan' 'Napoli'
 'Paris S-G' 'Porto' 'Rangers' 'RB Leipzig' 'RB Salzburg' 'Real Madrid'
 'Sevilla' 'Shakhtar' 'Sporting CP' 'Tottenham' 'Viktoria Plzeň'
 'Atalanta' 'Beşiktaş' 'Dynamo Kyiv' 'Lille' 'Malmö' 'Manchester Utd'
 'Sheriff Tiraspol' 'Villarreal' 'Wolfsburg' 'Young Boys' 'ruZen'
 'Başakşehir' 'Ferencváros' 'Krasnodar' 'Lazio' 'Loko Moscow' "M'Gladbach"
 'Midtjylland' 'Olympiacos' 'frRenn' 'Galatasaray' 'Genk' 'Lyon'
 'Red Star' 'Slavia Prague' 'Valencia' 'AEK Athens' 'CSKA Moscow'
 'Hoffenheim' 'Monaco' 'PSV Eindho

Vemos que hay muchos equipos diferentes, así que no vale la pena convertir esta columna a numérica. Sin embargo, el resto de columas sí que vale la pena convertirlas a numéricas.

In [9]:
# Columna 'Season'
le = LabelEncoder().fit(datos['Season'])
datos['Season'] = le.transform(datos['Season'])

# Obtenemos los valores únicos originales
valores_originales = le.classes_
# Creamos un diccionario de mapeo para ver a qué valor numérico corresponde cada valor original
temporada = dict(zip(valores_originales, le.transform(valores_originales)))

print(temporada)

{'2003-2004': 0, '2004-2005': 1, '2005-2006': 2, '2006-2007': 3, '2007-2008': 4, '2008-2009': 5, '2009-2010': 6, '2010-2011': 7, '2011-2012': 8, '2012-2013': 9, '2013-2014': 10, '2014-2015': 11, '2016-2017': 12, '2018-2019': 13, '2019-2020': 14, '2020-2021': 15, '2021-2022': 16, '2022-2023': 17}


In [10]:
# Columna 'Country'
le = LabelEncoder().fit(datos['Country'])
datos['Country'] = le.transform(datos['Country'])

# Obtenemos los valores únicos originales
valores_originales = le.classes_
# Creamos un diccionario de mapeo para ver a qué valor numérico corresponde cada valor original
paises = dict(zip(valores_originales, le.transform(valores_originales)))

print(paises)

{'Alemania': 0, 'Austria': 1, 'Bielorrusia': 2, 'Bulgaria': 3, 'Bélgica': 4, 'Chipre': 5, 'Croacia': 6, 'Dinamarca': 7, 'Escocia': 8, 'Eslovaquia': 9, 'Eslovenia': 10, 'España': 11, 'Francia': 12, 'Grecia': 13, 'Hungría': 14, 'Inglaterra': 15, 'Israel': 16, 'Italia': 17, 'Moldavia': 18, 'Noruega': 19, 'Países bajos': 20, 'Polonia': 21, 'Portugal': 22, 'República checa': 23, 'Rumanía': 24, 'Rusia': 25, 'Serbia': 26, 'Suecia': 27, 'Suiza': 28, 'Turquía': 29, 'Ucrania': 30}


Último paso de la ingeniería de variables, eliminamos variables categóricas y realizamos un escalado de características.

In [11]:
# Eliminamos la única columna no numérica que queda
datos = datos.drop(columns=['Squad'])

# Escalamos los datos para mejorar la efectividad de los algoritmos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(datos)

Aplicamos los algoritmos de clustering.

In [12]:
algorithms = {
    "KMeans": KMeans(n_clusters=3),
    "MeanShift": MeanShift(),
    "DBSCAN": DBSCAN(eps=0.5, min_samples=5),
    "OPTICS": OPTICS(),
    "GMM": GaussianMixture(n_components=3),
    "AgglomerativeClustering": AgglomerativeClustering(n_clusters=3)
}

Iteramos sobre varios algoritmos de clustering, aplicamos cada algoritmo al conjunto de datos, contamos el número de clusters encontrados por cada algoritmo y luego imprimimos esta información para cada algoritmo. Esto permite comparar cómo cada algoritmo agrupa los datos y cuántos clusters encuentra.

In [13]:
for name, algorithm in algorithms.items():
    cluster_labels = algorithm.fit_predict(X_scaled)
    if cluster_labels is not None:
        n_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)
        print(f"{name}: {n_clusters} clusters")
    else:
        print(f"Error: {name} algorithm returned None")


KMeans: 3 clusters


MeanShift: 1 clusters
DBSCAN: 0 clusters
OPTICS: 4 clusters
GMM: 3 clusters
AgglomerativeClustering: 3 clusters
